In [72]:
# import spotify api for python
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

### Connect to Spotify API

In [2]:
# credentials
client_id= ""
client_secret= ""
redirect_uri='http://google.com/'

In [3]:
# set scope
username='1257375947'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret) 
scope = 'user-library-read playlist-read-private'
try:
    token = util.prompt_for_user_token(username, scope,client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)  
    sp=spotipy.Spotify(auth= token)
except:
    print('Token is not accesible for ' + username)

### Pull Hip-Hop and Classical Playlist Data

In [47]:
# get playlists
songLibrary = sp.current_user_saved_tracks()
# hip-hop playlists
hh_playlist = sp.user_playlist('spotify', playlist_id='4ykrONBhOgPr4sMhAVCoPx') # Hip Hop Hits
hh_playlist2 = sp.user_playlist('spotify', playlist_id='37i9dQZF1DX9s3cYAeKW5d') # Hip-Hop Work Out Mix
hh_playlist3 = sp.user_playlist('spotify', playlist_id='37i9dQZF1DX6GwdWRQMQpq') # Feelin' Myself
hh_playlist4 = sp.user_playlist('spotify', playlist_id='37i9dQZF1DX48TTZL62Yht') # Hip Hop Favourites
hh_playlist5 = sp.user_playlist('spotify', playlist_id='37i9dQZF1DWUFmyho2wkQU') # Hip Hop Drive

# classical playlists
# cl_playlist = sp.user_playlist('spotify', playlist_id='37i9dQZF1DWWEJlAGA9gs0') # Classical Essentials
cl_playlist2 = sp.user_playlist('spotify', playlist_id='37i9dQZF1DXd5zUwdn6lPb') # Classical Focus
# cl_playlist3 = sp.user_playlist('spotify', playlist_id='37i9dQZF1DX4s3V2rTswzO') # Classical Romance
cl_playlist4 = sp.user_playlist('spotify', playlist_id='37i9dQZF1DWV0gynK7G6pD') # Classical New Releases

In [48]:
# get tracks
def get_tracks(playlist):
  tracks = playlist["tracks"]
  songs = tracks["items"] 
  while tracks['next']:
      tracks = sp.next(tracks)
      for item in tracks["items"]:
          songs.append(item)
  ids = [] 
  for i in range(len(songs)):
      ids.append(songs[i]['track']['id'])
  return(ids)

In [64]:
# Hip-hop ids
hh_ids = get_tracks(hh_playlist)
hh_ids2 = get_tracks(hh_playlist2)
hh_ids3 = get_tracks(hh_playlist3)
hh_ids4 = get_tracks(hh_playlist4)
hh_ids5 = get_tracks(hh_playlist5)

hh_ids_all = hh_ids+hh_ids2+hh_ids3+hh_ids4+hh_ids5

# Classical ids
# cl_ids = get_tracks(cl_playlist)
cl_ids2 = get_tracks(cl_playlist2)
# cl_ids3 = get_tracks(cl_playlist3)
cl_ids4 = get_tracks(cl_playlist4)

cl_ids_all = cl_ids2+cl_ids4#+cl_ids3+cl_ids4

In [65]:
# remove duplicates
def keep_unique(test_list):
    res = [] 
    for i in test_list: 
        if i not in res: 
            res.append(i)
    return(res)

In [66]:
# remove duplicates and check length --trying to keep classes balanced
hh_ids_uniq = keep_unique(hh_ids_all)
print('# of Hip-Hop tracks:', len(hh_ids_uniq))
cl_ids_uniq = keep_unique(cl_ids_all)
print('# of Classical Tracks:', len(cl_ids_uniq))

# of Hip-Hop tracks: 313
# of Classical Tracks: 293


In [67]:
# get audio features
def get_features(ids):
  features = []
  for i in range(0,len(ids),50):
      audio_features = sp.audio_features(ids[i:i+50])
      for track in audio_features:
          features.append(track)
          features[-1]['target'] = 1
  return(features)

In [68]:
# Hip-hop features
hh_features = get_features(hh_ids_uniq)

# Classical features
cl_features = get_features(cl_ids_uniq)

### Build dataframe and write to file

In [75]:
import pandas as pd
from sklearn.utils import shuffle

In [71]:
# Add new 'genre' column with Hip_hop and Classical
hh_features = pd.DataFrame(hh_features)
hh_features['genre'] = 'Hip-Hop'

cl_features = pd.DataFrame(cl_features)
cl_features['genre'] = 'Classical'

In [73]:
# make one dataframe and shuffle rows
features = hh_features.append(cl_features)
features = shuffle(features)

In [74]:
# save to .csv file to use for analysis
features.to_csv('track_data.csv', index=False)